# SETUP

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import psycopg2
from sqlalchemy import create_engine

# GET DATA

In [3]:
# fetch a web page
htlm_page = requests.get("https://www.francetvinfo.fr/economie/")
print(htlm_page.text[:1000])

<!DOCTYPE html><!--[if lt IE 7]><html class="no-js ie ie6 lte9 lte8 lte7" lang="fr"> <![endif]--><!--[if IE 7]><html class="no-js ie ie7 lte9 lte8 lte7" lang="fr"> <![endif]--><!--[if IE 8]><html class="no-js ie ie8 lte9 lte8" lang="fr"> <![endif]--><!--[if IE 9]><html class="no-js ie ie9 lte9" lang="fr"> <![endif]--><!--[if gt IE 9]><html class="no-js ie" lang="fr"> <![endif]--><!--[if !IE]><!--><html class="no-js" lang="fr"> <!--<![endif]--><head><title>Eco / Conso</title><meta http-equiv="pragma" content="no-cache" /><meta http-equiv="expires" content="Tue, 22 Feb 2000 12:14:57 GMT" /><meta http-equiv="refresh" content="1799" /><meta property="og:title" content="Eco / Conso" /><meta property="og:description" content="Suivez en direct toute l'actualité &quot;Eco / Conso&quot; : vivez l'info en live, en images et en vidéos. Participez, commentez et partager avec Franceinfo en temps réel !" /><meta property="og:type" content="website" /><meta property="og:locale:alternate" content="fr_

In [4]:
page = BeautifulSoup(htlm_page.text, "html5lib")
print(page.get_text()[:1000])

Eco / Conso  var myHash='',queryString=document.location.search;var xtref=document.referrer.replace(/[<>]/g,'').replace(/&/g,'$');myHash='xtref='+((''!=xtref)?xtref:'acc_dir');if(document.location.search&&(new RegExp("xtor=([^&]+)")).test(document.location.search)){var xtor=(new RegExp("xtor=([^&]+)")).exec(document.location.search);if(xtor){myHash=((myHash!='')?'&':'')+'xtor='+xtor[1]} console.log('document.location.search',document.location.search);var queryString=document.location.search.replace(xtor[0],'');console.log("queryString",queryString);} myHash=(document.location.hash!='')?document.location.hash+((myHash!='')?'&'+myHash:''):'#'+myHash;(function(mobileUri){var isIpad=navigator.userAgent.match(/iPad/i)!=null,isMobile="undefined"!==typeof window.screen&&"undefined"!==typeof window.screen.width&&(768>=window.screen.width);if(isMobile&&!isIpad){window.location.href="//"+window.location.hostname.replace("www","mobile")+mobileUri}})(("/economie/"+queryString+myHash));  (function(

## EXTRACT TITLES

In [5]:
# extract title
summaries = page.find_all('div', class_="flowItem")

In [6]:
summaries[0]

<div class="flowItem feature" entity="CMS-Article_3848977"><a href="/economie/retraite/reforme-des-retraites/reforme-des-retraites-francois-ruffin-demande-la-dissolution-de-l-assemblee-nationale-apres-le-recours-au-49-3_3848977.html"><figure id="image_3645841"><img alt="Le député LFI de la Somme François Ruffin sur les bancs de l\'Assemblée nationale le 17 février 2020." botag="CMS-IMAGE_3645841" class="b-lazy" data-src="/image/75rw29fwf-0048/840/472/21056517.jpg" height="472px" src="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==" title="Le député LFI de la Somme François Ruffin sur les bancs de l'Assemblée nationale le 17 février 2020. | CHRISTOPHE ARCHAMBAULT / AFP" width="840px"/></figure><div class="right-content"><h2 class="title" field="title">Réforme des retraites : François Ruffin demande la dissolution de l'Assemblée nationale après le recours au 49.3</h2><div class="description" field="description">Le député LFI de la Somme a estimé que les motions

In [7]:
summaries[0].select_one("h2").get_text()

"Réforme des retraites : François Ruffin demande la dissolution de l'Assemblée nationale après le recours au 49.3"

In [8]:
len(summaries)

28

In [10]:
eco_news = [x.select_one("h2").get_text() for x in summaries]

In [11]:
eco_news

["Réforme des retraites : François Ruffin demande la dissolution de l'Assemblée nationale après le recours au 49.3",
 'Coronavirus, 49.3, Adèle Haenel aux César... Le "8h30 franceinfo" de Sibeth Ndiaye',
 "Réforme des retraites : l'intersyndicale appelle à des mobilisations mardi partout en France contre le recours à l'article 49.3",
 'Le cinéaste Nicolas Philibert fait des films "pour apprendre"',
 'VIDEO. Réforme des retraites\xa0: Sibeth Ndiaye justifie le recours au 49.3 par un "impératif" lié aux délais réduits par les sénatoriales',
 'Le coronavirus grippe tourisme, hôtellerie et restauration à Paris',
 'Réforme des retraites : "Le recours au 49.3 est un scandale", estime Ian Brossat (PCF)',
 "Réforme des retraites : malgré le recours au 49.3, les opposants n'ont pas abdiqué",
 '"Nous débloquerons ce qu\'il faudra" pour les entreprises pénalisées par le coronavirus, assure Bruno Le Maire',
 'VIDEO. Réforme des retraites : Jean-Luc Mélenchon appelle à des mobilisations populaires 

# WRITE THE DATA IN POSTGRESQL DB

In [12]:
df = pd.DataFrame({"source": "franceinfo", "headline": eco_news, "date": pd.to_datetime('today').to_pydatetime().date()})

In [13]:
df.head()

,source,headline,date
0,franceinfo,Réforme des retraites : François Ruffin demand...,2020-03-02
1,franceinfo,"Coronavirus, 49.3, Adèle Haenel aux César... L...",2020-03-02
2,franceinfo,Réforme des retraites : l'intersyndicale appel...,2020-03-02
3,franceinfo,"Le cinéaste Nicolas Philibert fait des films ""...",2020-03-02
4,franceinfo,VIDEO. Réforme des retraites : Sibeth Ndiaye j...,2020-03-02


In [14]:
# create an engine
alchemy_engine = create_engine('postgresql+psycopg2://antoinemertz@localhost/news', pool_recycle=3600)

In [15]:
conn = alchemy_engine.connect()
table = "economic"

In [16]:
try:
    frame = df.to_sql(name=table, con=conn, if_exists='append', index=False)
except Exception as ex:
    print(ex)
conn.close()